# Create image log mesh using Open3D Poisson Surface Reconstruction Algorithm

In [ ]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt

## Load wellbore image data

In [ ]:
fmi_raster = np.array([i for i in np.load('fmi_raster.npy')[3500000:-1,:]])
fmi_xyz = np.array([i for i in np.load('fmi_xyz.npy')[3500000:-1,:]])
fmi_xyz_n = np.array([i for i in np.load('fmi_xyz_n.npy')[3500000:-1,:]])

## Preprocessing by creating point cloud data from the wellbore image arrays

In [ ]:
# Pass xyz, RBG and normals to Open3D.o3d.geometry.PointCloud and visualize
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(fmi_xyz)
pcd.colors = o3d.utility.Vector3dVector(fmi_raster[:,:3])
pcd.normals = o3d.utility.Vector3dVector(fmi_xyz_n)
o3d.visualization.draw_geometries([pcd], point_show_normal=True)

## Poisson surface reconstruction

In [ ]:
# runing Poisson surface reconstruction
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=10)
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

In [ ]:
#  visualize mesh vertrex densities
densities = np.asarray(densities)
density_colors = plt.get_cmap('plasma')((densities - densities.min()) / (densities.max() - densities.min()))
density_colors = density_colors[:, :3]
density_mesh = o3d.geometry.TriangleMesh()
density_mesh.vertices = mesh.vertices
density_mesh.triangles = mesh.triangles
density_mesh.triangle_normals = mesh.triangle_normals
density_mesh.vertex_colors = o3d.utility.Vector3dVector(density_colors)
o3d.visualization.draw_geometries([density_mesh], mesh_show_back_face=True)

In [ ]:
# removing low density vertices
vertices_to_remove = densities < np.quantile(densities, 0.1)
mesh.remove_vertices_by_mask(vertices_to_remove)
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)
o3d.io.write_triangle_mesh("mesh.ply", mesh)